## 4.1 将LLM接入LangChain

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('.env.local'))

api_key = os.getenv("DEEPSEEK_API_KEY")
api_url = os.getenv("DEEPSEEK_API_URL")
model = os.getenv("DEEPSEEK_MODEL")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=api_key,
    base_url=api_url,
    model_name=model,
    temperature=0.0,
)

/Users/young/project/llmProject/llm-universe/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
output = llm.invoke("请你自我介绍一下自己！")

In [5]:
output

AIMessage(content='你好呀！我是 **DeepSeek Chat**，由深度求索（DeepSeek）公司打造的智能 AI 助手。我的最新版本是 **DeepSeek-V3**，知识截止到 **2024年7月**，拥有 **128K 上下文记忆**，可以处理超长文本，还能阅读和解析 **PDF、Word、Excel、PPT、TXT** 等文件内容。  \n\n### **我的特点：**  \n🔹 **免费使用**：目前不收费，随时为你解答问题！  \n🔹 **超长上下文**：支持长达 128K 的对话记忆，适合处理复杂任务。  \n🔹 **文件阅读**：可以帮你分析文档内容，提取关键信息。  \n🔹 **知识丰富**：覆盖科技、编程、学习、生活、娱乐等多个领域。  \n🔹 **逻辑清晰**：擅长推理、写作、翻译、代码编写等任务。  \n\n无论是学习、工作，还是日常生活中的疑问，都可以来找我聊聊！😊 你今天有什么想了解的呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 218, 'prompt_tokens': 8, 'total_tokens': 226, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'finish_reason': 'stop', 'logprobs': None}, id='run-1553582b-f98b-473b-9a15-1a507ab9ad7b-0', usage_metadata={'input_tokens': 8, 'output_tokens': 218, 'total_tokens': 226})

In [8]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

In [9]:
# 使用prompt模板
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([
    ("system", template),
    ("human", human_template)
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"

messages = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

ChatPromptValue(messages=[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})])

In [10]:
# 测试一下
output = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.  \n\n(Note: This translation aims to preserve the poetic and somewhat surreal imagery of the original while ensuring clarity in English. The phrase "比身体重的行李" is rendered as "luggage heavier than my body" to convey both literal and metaphorical weight. "游入尼罗河底" becomes "Dived into the depths of the Nile" to maintain the sense of submersion and mystery. "光圈" is translated as "halos" to evoke a luminous, almost otherworldly visual, fitting the dreamlike tone.)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 45, 'total_tokens': 195, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 45}, 'model_name': 'deepseek-chat'

In [11]:
# 格式化输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.  \n\n(Note: This translation aims to preserve the poetic and somewhat surreal imagery of the original while ensuring clarity in English. The phrase "比身体重的行李" is rendered as "luggage heavier than my body" to convey both literal and metaphorical weight. "游入尼罗河底" becomes "Dived into the depths of the Nile" to maintain the sense of submersion and mystery. "光圈" is translated as "halos" to evoke a luminous, almost otherworldly visual, fitting the dreamlike tone.)'

In [12]:
# 使用LCEL表达式
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language": "中文", "output_language": "英文", "text": text})

'I carried luggage heavier than my body,  \nDived into the depths of the Nile,  \nThrough several flashes of lightning,  \nI saw a cluster of halos—  \nUnsure if this was the place.'

In [13]:
# 再试一下，英译中
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'我扛着比身体还重的行李潜入尼罗河底，穿过几道闪电后看见一堆光晕，不知是不是这里。'

## 4.2 构建检索问答链

### 4.2.1 加载向量数据库

In [15]:
from ark_embedding import ArkEmbeddings
from langchain.vectorstores.chroma import Chroma

import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv('.env.local'))

embedding_api_key = os.getenv("ARK_API_KEY")
embedding_api_url = os.getenv("ARK_API_URL")
embedding_model = os.getenv("ARK_EMBEDDING_MODEL")

In [16]:
# 初始化 Embeddings
embedding = ArkEmbeddings(
    api_key=embedding_api_key,
    api_url=embedding_api_url,
    model=embedding_model,
)

# 向量数据库持久化路径
persist_directory = "../data_base/vector_db/chroma"

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

/var/folders/7r/h44tfn_x35jg4s7msmb1gdzw0000gp/T/ipykernel_7361/801487602.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(


In [17]:
print(f"向量库中存储的数据：{vectordb._collection.count()}")

向量库中存储的数据：1004


In [18]:
# 检索相似文档
question = "什么是prompt engineering?"
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


In [19]:
# 打印检索内容
for i, doc in enumerate(docs):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第1个内容: 
 具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调整试错，最终找到可靠适用的 Prompt 形式才是设计 Prompt 的正确方法。

读者可以在 Jupyter Notebook 上，对本章给出的示例进行实践，修改 Prompt 并观察不同输出，以深入理解 Prompt 迭代优化的过程。这会对进一步开发复杂语言模型应用提供很好的实践准备。

三、英文版

产品说明书
-----------------------------------------------------
检索到的第2个内容: 
 第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large Language Model， 大语言模型）技术在产品中的应用方面做出了很大贡献。她还参与编写了教授人们使用 Prompt 的 OpenAI cookbook。我们希望通过本模块的学习，与大家分享使用提示词开发 LLM 应用的最佳实践和技巧。
-----------------------------------------------------
检索到的第3个内容: 
 第二章 提示原则

如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。

首先，Prom

### 4.2.2 创建检索链

In [20]:
from langchain_core.runnables import RunnableLambda

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("南瓜书是什么？")

'5.1 综合样例\n\n最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会\n主编：Sm1les、archwalker、jbb0523\n编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226\n封面设计：构思-Sm1les、创作-林王茂盛\n致谢\n特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、\nspareribs、sunchaothu、StevenLzq 在最早期的时候对南瓜书所做的贡献。\n扫描下方二维码，然后回复关键词“南瓜书”，即可加入“南瓜书读者交流群”\n版权声明\n本作品采用知识共享署名-非商业性使用-相同方式共享4.0 国际许可协议进行许可。\n\n←_←'

### 4.2.3 创建LLM

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=api_key,
    base_url=api_url,
    model_name=model,
    temperature=0.0,
)

llm.invoke("请你自我介绍一下自己！").content

'你好呀！我是 **DeepSeek Chat**，由深度求索公司（DeepSeek）研发的一款智能AI助手。我的核心版本是 **DeepSeek-V3**，知识截止到 **2024年7月**，拥有 **128K 上下文记忆**，可以处理超长文本，还能阅读 **PDF、Word、Excel、PPT、TXT** 等文件，帮助你高效获取信息！  \n\n✨ **我的特点**：  \n- **免费使用**：目前不收费，随时为你解答问题！  \n- **强大的理解与创作能力**：无论是学习、写作、编程、办公还是生活建议，我都能帮上忙。  \n- **超长上下文支持**：可以记住更长的对话内容，适合处理复杂任务。  \n- **文件阅读**：上传文档，我可以帮你总结、提取关键信息或分析内容。  \n\n如果你有任何问题，无论是学术研究、代码编写、文案创作，还是日常闲聊，都可以找我聊聊！😊 你今天想了解什么呢？'

### 4.2.4 构建检索问答链

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 让模型根据检索的结果，回答问题
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题: {input}
"""

# 将template通过 PromptTemplate 转为可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

# 处理流程
# 1. 获取检索结果，并保留原有的输入
# 2. 拼装prompt
# 3. 请求llm
# 4. 格式化输出
qa_chain = (
    RunnableParallel({"context": retrieval_chain, "input": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
# 使用知识库
# 效果测试
question_1 = "什么是南瓜书？"
question_2 = "Prompt Engineering for Developer是谁写的？"

In [24]:
result = qa_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

大模型+知识库后回答 question_1 的结果：
南瓜书是一本由Datawhale社区编撰的书籍，最新版PDF可通过GitHub获取。它由多位编委和贡献者共同完成，采用知识共享许可协议发布。谢谢你的提问！


In [25]:
result = qa_chain.invoke(question_2)
print("大模型+知识库后回答 question_2 的结果：")
print(result)

大模型+知识库后回答 question_2 的结果：
《Prompt Engineering for Developer》是由吴恩达与OpenAI的Isa Fulford合作编写的。谢谢你的提问！


In [26]:
# 不使用知识库
# 效果测试
llm.invoke(question_1).content

'**南瓜书（PumpkinBook）** 是《机器学习》（西瓜书，周志华著）的**公式推导补充手册**，由开源学习社区Datawhale团队发起编写。它旨在帮助读者更深入地理解西瓜书中因篇幅限制未详细展开的数学推导和难点，尤其适合数学基础较弱的读者补充学习。\n\n---\n\n### **关键信息：**\n1. **名称由来**  \n   - 《机器学习》因封面有西瓜被昵称“西瓜书”，而配套的推导手册延续这一风格，取名“南瓜书”（Pumpkin Book）。\n\n2. **核心内容**  \n   - 对西瓜书中**关键公式的详细推导**（如支持向量机、神经网络、概率图模型等章节的数学细节）。\n   - 补充相关数学知识（如矩阵求导、概率论等），降低学习门槛。\n\n3. **特点**  \n   - **开源免费**：项目托管在GitHub，可随时查阅或贡献。\n   - **社区驱动**：由多名志愿者合作完成，持续迭代更新。\n   - **配套资源**：部分版本提供习题解析或代码实现。\n\n4. **适用人群**  \n   - 机器学习初学者，尤其是数学基础薄弱者。\n   - 希望深入理解西瓜书理论细节的读者。\n\n---\n\n### **获取方式：**\n- **GitHub仓库**：搜索 `datawhalechina/pumpkin-book` 可找到最新版本。\n- **PDF下载**：通常可在仓库的Release页面或社区论坛获取。\n\n---\n\n### **与其他资料的关系：**\n- 南瓜书与《机器学习》（西瓜书）**配套使用**，而非独立教材。\n- 类似资源还有《西瓜书习题解答》等，但南瓜书更聚焦公式推导。\n\n如果需要具体章节的推导示例或进一步解读，可以告知具体问题哦！'

In [27]:
llm.invoke(question_2).content

'《Prompt Engineering for Developer》是由 **DeepLearning.AI** 与 **OpenAI** 合作推出的课程内容，主要作者和讲师是 **Andrew Ng（吴恩达）** 和 **Isa Fulford（OpenAI 的技术团队成员）**。  \n\n### 关键点：\n1. **Andrew Ng** 是 DeepLearning.AI 的创始人，也是机器学习领域的知名教育家（曾任斯坦福大学教授、Coursera 联合创始人）。  \n2. **Isa Fulford** 是 OpenAI 的开发者关系工程师，专注于帮助开发者有效使用 OpenAI 的 API 和工具。  \n3. 这门课程是 **免费短期课程**，专注于教授开发者如何通过提示工程（Prompt Engineering）优化大语言模型（如 ChatGPT）的应用。  \n\n### 课程内容：\n- 基础提示设计技巧  \n- 迭代优化提示的方法  \n- 构建实际应用（如聊天机器人、文本摘要等）  \n\n课程可通过 DeepLearning.AI 官网或 Coursera 平台访问。如果需要链接或进一步信息，可以告诉我！'

- 从上述的结果来看，增加了知识库，LLM的回答效果，并没有明显的变好，甚至有一些劣化，
- 可能是因为这个问题有些旧了，且专业性不够强。
- 也有可能是prompt里，对回复的长度进行了强限制。

### 4.2.5 向检索链添加聊天记录

In [29]:
from langchain_core.prompts import ChatPromptTemplate

# 问答链的系统prompt
system_prompt = (
    "你是一个问答任务的助手。 "
    "请使用检索到的上下文片段回答这个问题。 "
    "如果你不知道答案就说不知道。 "
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)

# 制定prompt template
qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [30]:
# 无历史记录
messages = qa_prompt.invoke(
    {
        "input": "南瓜书是什么？",
        "chat_history": [],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


南瓜书是什么？


In [31]:
# 有历史记录
messages = qa_prompt.invoke(
    {
        "input": "你可以介绍一下他吗？",
        "chat_history": [
            ("human", "西瓜书是什么？"),
            ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
        ],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。 请使用检索到的上下文片段回答这个问题。 如果你不知道答案就说不知道。 请使用简洁的话语回答用户。


西瓜书是什么？
西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。
你可以介绍一下他吗？


- 可以看到，上述的方案，只是简单地将历史的文本和当前的问题进行拼接。

### 4.2.6 带有信息压缩的检索链

In [46]:
from langchain_core.runnables import RunnableBranch

# 压缩问题的系统 prompt
condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    "确保最后的输出是一个完整的问题。"  # NOTE: 这里需要确保最后的输出是一个完整的问题
)

# 构建压缩问题的 prompt template
condense_question_prompt = ChatPromptTemplate([
    ("system", condense_question_system_template),
    ("placeholder", "{chat_history}"),
    ("human", "{input}")
])

# 构造“总结历史信息”的检索文档的处理链
# RunnableBranch 会根据条件选择要运行的分支
retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", ""), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

In [ ]:
# 原问题
test_chain = condense_question_prompt | llm | StrOutputParser()
test_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [],
})

'南瓜书跟它有什么关系？'

In [ ]:
# 完善信息后的问题
test_chain = condense_question_prompt | llm | StrOutputParser()
test_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ],
})

'南瓜书和西瓜书有什么关系？'

In [47]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"])

# 定义“整合知识库”的问答链
# 1. 整合知识库信息进入context
# 2. 拼装prompt, 整合context和chat_history进入qa_chain
# 3. 请求llm
# 4. 格式化输出
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数，整合知识库的内容得到context输入 qa_prompt
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)

# 定义带有历史记录的问答链
# 1. 检索知识库(结合总结历史信息), 并存入context
# 2. 整合context和chat_history进入qa_chain
qa_history_chain = RunnablePassthrough.assign(
    context=retrieve_docs # 将查询结果存为 context
).assign(answer=qa_chain) # 将最终结果存为 answer

- chat_history在两个地方使用了:
- 1.检索知识库时；
- 2.在请求llm时。

In [48]:
# 测试检索问答链
# 不带聊天记录
qa_history_chain.invoke({
    "input": "西瓜书是什么？",
    "chat_history": []
})

{'input': '西瓜书是什么？',
 'chat_history': [],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 13, 'producer': 'xdvipdfmx (20200315)', 'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←\n第1 章\n绪论\n本章作为“西瓜书”的开篇，主要讲解什么是机器学习以及机器学习的相关数学符号，为后续内容作\n铺垫，并未涉及复杂的算法理论，因此阅读本章时只需耐心梳理清楚所有概念和数学符号即可。此外，在\n阅读本章前建议先阅读西瓜书目录前页的《主要符号表》，它能解答在阅读“西瓜书”过程中产生的大部\n分对数学符号的疑惑。\n本章也作为本书的开篇，笔者在此赘述一下本书的撰写初衷，本书旨在以“过来人”的视角陪读者一\n起阅读“西瓜书”，尽力帮读者消除阅读过程中的“数学恐惧”，只要读者学习过《高等数学》、《线性代\n数》和《概率论与数理统计》这三门大学必修的数学课，均能看懂本书对西瓜书中的公式所做的解释和推\n导，同时也能体会到这三门数学课在机器学习上碰撞产生的“数学之美”。\n1.1\n引言\n本节以概念理解为主，在此对“算法”和“模型”作补充说明。“算法”是指从数据中学得“模型”的具\n体方法，例如后续章节中将会讲述的线性回归、对数几率回归、决策树等。“算法”产出的结果称为“模型”，'),
  Document(metadata={'autho

In [49]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ]
})

{'input': '南瓜书跟它有什么关系？',
 'chat_history': [('human', '西瓜书是什么？'),
  ('ai', '西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。')],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 13, 'producer': 'xdvipdfmx (20200315)', 'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←\n第1 章\n绪论\n本章作为“西瓜书”的开篇，主要讲解什么是机器学习以及机器学习的相关数学符号，为后续内容作\n铺垫，并未涉及复杂的算法理论，因此阅读本章时只需耐心梳理清楚所有概念和数学符号即可。此外，在\n阅读本章前建议先阅读西瓜书目录前页的《主要符号表》，它能解答在阅读“西瓜书”过程中产生的大部\n分对数学符号的疑惑。\n本章也作为本书的开篇，笔者在此赘述一下本书的撰写初衷，本书旨在以“过来人”的视角陪读者一\n起阅读“西瓜书”，尽力帮读者消除阅读过程中的“数学恐惧”，只要读者学习过《高等数学》、《线性代\n数》和《概率论与数理统计》这三门大学必修的数学课，均能看懂本书对西瓜书中的公式所做的解释和推\n导，同时也能体会到这三门数学课在机器学习上碰撞产生的“数学之美”。\n1.1\n引言\n本节以概念理解为主，在此对“算法”和“模型”作补充说明。“算法”是指从数据中学得“模型”的具\n体方法，例

## 4.3 部署知识库助手